# 전처리(1) - 맞춤법 검사 및 오탈자 교정
- 최종 speller : py-hanspell
- 개인 jupyter notebook 환경에서 install 및 import 가능함 (이외 환경에서는 에러 발생)

## [환경 설정]
- 텐서플로우, 토치 설치
- import

In [3]:
# !pip install tensorflow

In [5]:
# !pip install torch

In [15]:
import tensorflow as tf
print("TensorFlow name : ", tf.__name__)

TensorFlow name :  tensorflow


In [161]:
import os
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
import numpy as np
import time
import csv

print("다음 단계로")

다음 단계로


#### [기본] speller(1) py-hanspell import

In [60]:
# speller(1) py-hanspell import
### 로컬 환경에서 실행
### py-hanspell이 프로젝트 폴더 하위 폴더여야 함 

# 제일 최근 업데이트 pull-request (아직 merge 안됨)
!pip install git+https://github.com/drtagkim/py-hanspell.git 

In [229]:
from hanspell import spell_checker  # py_hanspell

In [ ]:
# 샘플 코드
# result = spell_checker.check(u'태권도를 처읆 배워')
# result.as_dict()

#### [추가] speller(2) BaseOnPLMmodelET5 import (생략 가능)

In [ ]:
# [생략 가능]  
# speller(2) BaseOnPLMmodelET5 import 
### test 해 볼 경우에만 import
### colab 환경이어야 함 

import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
''' error 설명 : " Error displaying widget "
주피터 노트북 환경에서 실행하니 에러 메시지 발생하였지만, 실행에 문제가 되지 않음
코랩에서는 이러한 에러 및 에러 메시지 발생하지 않음
'''

# T5 모델 로드
model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")
tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "mps:0" if torch.cuda.is_available() else "cpu" # for mac m1
model = model.to(device)

## [data] data 준비 및 label 인코딩 

In [86]:
# train data
# train_data_path ="~/aiffel/dktc/data/train.csv"

# train_data_path = "/content/drive/MyDrive/aiffel/dktc/data/train.csv"
train_data_path = "D:/DLThon/맞춤법교정/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [88]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3950 non-null   int64 
 1   class         3950 non-null   object
 2   conversation  3950 non-null   object
dtypes: int64(1), object(2)
memory usage: 92.7+ KB


In [90]:
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])

corpus = train_data["conversation"]

In [92]:
corpus.head()

0    지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
Name: conversation, dtype: object

In [94]:
# 일반 대화 데이터셋
# conversations_casual_path = '/content/drive/MyDrive/aiffel/dktc/data/sampled_dataset.csv'  # text file
conversations_casual_path = 'D:/DLThon/맞춤법교정/normal_conversation.csv'

# 인코딩을 명시적으로 지정하여 파일 읽기
data = pd.read_csv(conversations_casual_path, encoding='utf-8-sig')  # UTF-8 BOM(Byte Order Mark) 파일 읽어 옴

In [95]:
# 샘플링 4000개 
casual_data = data.sample(n=4000, random_state=42)

# 데이터 확인
print(casual_data.head(10))

                                            conversation  class
2574   여보 오늘 갔던 데에서 학원 많이 봤다고 하지 않았어요? 오 무슨 학원이 많았는데 ...  일반 대화
7495   나도 아파트에 한번 살아보고 싶다 키키 아파트 좋지~ 집값이 장난이 아니라 나는 이...  일반 대화
12230  넌 계속 부산에서 살 거냐 나는 부산 안 떠나고 싶어 ㅠㅠ 부산은 너 싫다는데? 하...  일반 대화
13920  나 며칠 전에 샹치 보고 왔음 오 그거 마블 새 영화? 응 맞아 첫 번째 아시안 히...  일반 대화
736    우리나라 여론은 통일을 안 해도 된다고 생각하는 사람들이 많잖아 우리 애들도 그렇게...  일반 대화
13073  서울 전세 값이 많이 올라서 걱정이야 ㅜㅜ 그러게... 서울도 서울인데 창원도 요새...  일반 대화
856    이제 백신접종 2차까지 마친 사람들은 12시까지 식당이나 술집을 이용할 수 있다네 ...  일반 대화
7272   아 자격증 공부 해야 하는디 요즘 넘 안 해 훈련도 하나의 교육이라 생각하면 편하지...  일반 대화
11496  오늘 길 가는데 너 선임 만났음 키키 엥? 어디서 걔를 만나? 키키 영화 보고 나오...  일반 대화
10619  제주도 여행 가고 싶어요 이번에 가게 되면 코스를 잘 짜고 싶다 코스도 코슨데 식당...  일반 대화


In [98]:
casual_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, 2574 to 4983
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   conversation  4000 non-null   object
 1   class         4000 non-null   object
dtypes: object(2)
memory usage: 93.8+ KB


## [공통] - 정규표현식, 맞춤법 검사기 함수 정의
- 정규표현식 : 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'의 반복을 각각 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'로 통일
- 최종 맞춤법 검사기 API : py-hanspell (비교 위해 ET5 기반 모델을 함께 제시함)

In [100]:
# 정규표현식 적용 함수

def preprocess_sentence(sentence):     # 추후 사용자 지정 단어 사전 구축하기 : 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^', '넹넹', '웅웅' 등
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'의 반복을 각각 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'로 통일
    sentence = re.sub(r"(ㅠ)+", "ㅠㅠ", sentence)  # 'ㅠㅠ' 이상 반복을 'ㅠㅠ'로
    sentence = re.sub(r"(ㅎ)+", "ㅎㅎ", sentence)  # 'ㅎㅎ' 이상 반복을 'ㅎㅎ'로
    sentence = re.sub(r"(ㅋ)+", "ㅋㅋ", sentence)  # 'ㅋㅋ' 이상 반복을 'ㅋㅋ'로
    # 한글, 알파벳, 문장 부호 및 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^'는 남기고 나머지는 공백으로 대체
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,ㅠㅠㅎㅎㅋㅋ^^]+", " ", sentence)

    sentence = sentence.strip()
    return sentence


def load_conversations_from_df(corpus_df):
    conv_data = []

    for index, row in corpus_df.iterrows():  # 수정된 부분
        conversations = row['conversation']  # 수정된 부분
        conv_data.append(preprocess_sentence(conversations))  # 전처리 함수 적용
    return conv_data

### 맞춤법 검사기 함수화 : py-hanspell, ET5 기반 speller API 

In [102]:
def textLine_from_row(text_from_row) :  # 각 row 데이터 -> 하나의 리스트, 발화 단위 분리 (각 원소 = 하나의 발화)
    text_line = []
    for i in range(0,text_from_row.size) :
        text = text_from_row.iloc[i, 0].split('\n')
        text_line.append(text)
    return text_line


def py_hanspell_spell_correct(text_line) :  # 분리된 각 line의 여러 문장을 읽어 옴 (하나의 발화 단위 text)
    checked_data_1 = []
    print('py-hanspell 텍스트 교정 시작')
    start = time.time()

    for i in range(len(text_line)) :
        for k in range(len(text_line[i])):  # 해당 인덱스 내 리스트/문자열을 순회
            result = spell_checker.check(f'{text_line[i][k]}').checked  # 맞춤법 검사
            checked_data_1.append(result)  # 결과 저장

    end = time.time()
    length = end - start
    print(length, '초 걸렸습니다.')
    return checked_data_1


def BaseOnPLMmodelET5_spell_correct(text_line) :  # 분리된 각 line의 여러 문장을 읽어 옴 (하나의 발화 단위 text)
    checked_data_2 = []
    print('BaseOnPLMmodelET5 텍스트 교정 시작')
    start = time.time()

    for i in range(len(text_line)) :
        for k in range(len(text_line[i])):  # 해당 인덱스 내 리스트/문자열을 순회
            result = typos_corrector(f'{text_line[i][k]}') # 맞춤법 검사
            checked_data_2.append(result)  # 결과 저장

    end = time.time()
    length = end - start
    print(length, '초 걸렸습니다.')
    return checked_data_2


In [199]:
def dataframe_to_csv(df, filename='output.csv'):   # 필터링 된 파일 저장 시 사용  
    # DataFrame을 CSV 파일로 저장
    df.to_csv(filename, index=False)  # 인덱스 제외하고 CSV로 저장
    print(f"DataFrame has been saved as {filename}")
    
    # Jupyter/Colab 환경에서 파일 다운로드 링크 제공
    try:
        from google.colab import files
        files.download(filename)
    except ImportError:
        print(f"File saved as {filename}. You can manually download it from the file system.")


In [201]:
# 누락 데이터 count 위한 함수 정의 

def convert_empty_to_nan(df):
    # DataFrame 내의 빈 문자열을 NaN으로 변환
    df.replace('', np.nan, inplace=True)
    return df

def count_empty_rows(df):
    # 각 행의 모든 값이 NaN인 경우를 찾음
    empty_rows = df.isnull().all(axis=1)
    
    # 빈 행의 개수 출력
    num_empty_rows = empty_rows.sum()
    return num_empty_rows

## [1-1] train 데이터셋 전처리

In [104]:
conv_data = load_conversations_from_df(train_data)

print('전체 샘플 수:', len(conv_data))

전체 샘플 수: 3950


In [105]:
# 한국어 전처리 함수 (정규표현식) 적용 결과

print('전처리 후의 1081번째 샘플: {}'.format(conv_data[1081]))
print('전처리 후의 500번째 샘플: {}'.format(conv_data[500]))

전처리 후의 1081번째 샘플: 아까 시에 배민에서 스테이크 덮밥 시켰던 사람인데요 시요 ? 벌써 시간이나 지났는데 . 무슨 일이신데요 ? 손님한테 그렇게 밖에 말을 못해요 ? 아닙니다 죄송합니다 오늘 스테이크에 제대로 된 고기 쓰시는 것 맞아요 ? 네 저희는 제대로 수입된 고기 납품받아 조리하고 있습니다 . 고기가 너무 질겨서 아이가 씹지도 못했어요 . 이거 환불해주세요 . 손님 배달 특성상 조리 후에 바로 배달된다해도 고기의 육질상태가 시간이 흐름에 따라 저하되는 것은 당연합니다 . 지금 말 다했어요 ? 그리고 반 이상 드셔놓고 환불해달라고 하시면 어떡해요 . 그럼 지금 고객인 내가 잘못했다는 거에요 ? 이거 안되겠네 . 당장 업체 본사에 전화하고 배민 고객센터에도 다 신고할거에요 . 당신 이 바닥에서 장사 못 해 이제 .
전처리 후의 500번째 샘플: 자기야 나 필요한거 있는뎅 . ! 어떤거 ? 음 나 가방 필요해서 그런데 만원만 주랑 웅 . ? 갑자기 ? 자기 며칠 전에도 가방 사줬잖아 그거는 내 생일선물이였구우 이번꺼는 그냥 여자친구한테 선물 줄 수 있는거잖아앙 ! 음자기가 이런식으로 매년 나한테 요구하는 돈이 얼만지 알아 ? 년마다 단위인데 우리 미래 생각하려면 돈 아껴야지 . ! 헐 모야잉 자기야 어차피 결혼하면 자기 돈 내 돈 상관 없이 같이 쓸텐데 나한테 쓰는게 아까워 ? ! 가방 살 돈 안주면 화낸다 ? ! 그게 아니라 그럼 사죠 돈 보내놔 알겠징 ? ! 자기야 사랑행 하 그래


### train data를 개별 class data로 분리 

In [121]:
print(train_data['class'])

0       3
1       3
2       1
3       0
4       0
       ..
3945    1
3946    0
3947    2
3948    0
3949    2
Name: class, Length: 3950, dtype: int32


In [123]:
def train_class_split(train_data) :  # data를 class 별로 분리
  train_classes = [pd.DataFrame() for _ in range(4)]

  for i in range(4) :
    train_classes[i] = train_data[train_data['class'] == i]

  return train_classes

In [125]:
train_classes = train_class_split(train_data)

In [127]:
train_classes[0].head()

,idx,class,conversation
3,3,0,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,0,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
9,9,0,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...
10,10,0,네 뭐 드릴까요?\n말보르 레드 하나.\n네. 여기요!\n여기 사장은 없어?\n네 ...
13,13,0,형! 형! 돈좀있어?\n저 저요.?\n여기 형말고 누가 있어? 형 얼마 있어?\n저...


In [129]:
conversations = []

for i in range(len(train_classes)):  
    conversations.append(train_classes[i]['conversation'])

# 각 클래스의 대화 데이터를 conversations에 저장
conversations_class_0 = conversations[0]
conversations_class_1 = conversations[1]
conversations_class_2 = conversations[2]
conversations_class_3 = conversations[3]

In [207]:
conversations_class_0.head()

3     어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4     저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
9     자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...
10    네 뭐 드릴까요?\n말보르 레드 하나.\n네. 여기요!\n여기 사장은 없어?\n네 ...
13    형! 형! 돈좀있어?\n저 저요.?\n여기 형말고 누가 있어? 형 얼마 있어?\n저...
Name: conversation, dtype: object

In [209]:
conv_c_0 = conversations_class_0.to_list()

In [211]:
conv_c_1 = conversations_class_1.to_list()

In [213]:
conv_c_2 = conversations_class_2.to_list()

In [215]:
conv_c_3 = conversations_class_3.to_list()

### 각 class data : speller 적용 

In [ ]:
py_hanspell_text_c_0 = py_hanspell_spell_correct(conv_c_0)

py-hanspell 텍스트 교정 시작


In [ ]:
py_hanspell_text_c_1 = py_hanspell_spell_correct(conv_c_1)

In [ ]:
py_hanspell_text_c_2 = py_hanspell_spell_correct(conv_c_2)

In [ ]:
py_hanspell_text_c_3 = py_hanspell_spell_correct(conv_c_3)

In [ ]:
# 교정 데이터 저장 
df_py_c_0 = pd.DafaFrame(py_hanspell_text_c_0)
df_py_c_1 = pd.DafaFrame(py_hanspell_text_c_1)
df_py_c_2 = pd.DafaFrame(py_hanspell_text_c_2)
df_py_c_3 = pd.DafaFrame(py_hanspell_text_c_3)

In [ ]:
### 누락 데이터 count 

df_p_c = [df_py_c_0, df_py_c_1, df_py_c_2, df_py_c_3]

for i in range(0, len(df_p_c)) :
    df = df_py_c[i].copy()
    df_p_c = convert_empty_to_nan(df)
    n = count_empty_rows(df_p_c)
    print(f'{n}개 행이 누락되었습니다.')


#### 각 클래스별 데이터프레임 csv로 다운로드 

In [ ]:
for i in range(0, len(df_p_c)) :
    dataframe_to_csv(df_p_c[i], f'C:/Users/ParkYeonwoo/Downloads/filtered_conv_{i}.csv')

In [ ]:
# 각 DataFrame을 CSV 파일로 저장
for i, df in enumerate(train_classes):
    df.to_csv(f'train_class_{i}.csv', index=False)

# 각 CSV 파일에 대한 다운로드 링크 제공 (Jupyter 환경에서 실행 시)
from IPython.display import FileLink, display

for i in range(len(train_classes)):
    display(FileLink(f'train_class_{i}.csv')) # 링크 에러시, 코랩 폴더 열면, 파일이 생성되어 있을 것임


## [1-2] 일반 대화 데이터셋 전처리

In [135]:
# 정규표현식 적용
preprocessed_casual_data = casual_data.copy()

for i in range(len(preprocessed_casual_data)):
    preprocessed_casual_data.iloc[i, 0] = preprocess_sentence(preprocessed_casual_data.iloc[i, 0])

preprocessed_casual_data.head(10)


,conversation,class
2574,여보 오늘 갔던 데에서 학원 많이 봤다고 하지 않았어요 ? 오 무슨 학원이 많았는데...,일반 대화
7495,나도 아파트에 한번 살아보고 싶다 키키 아파트 좋지 집값이 장난이 아니라 나는 이번...,일반 대화
12230,넌 계속 부산에서 살 거냐 나는 부산 안 떠나고 싶어 ㅠㅠ 부산은 너 싫다는데 ? ...,일반 대화
13920,나 며칠 전에 샹치 보고 왔음 오 그거 마블 새 영화 ? 응 맞아 첫 번째 아시안 ...,일반 대화
736,우리나라 여론은 통일을 안 해도 된다고 생각하는 사람들이 많잖아 우리 애들도 그렇게...,일반 대화
13073,서울 전세 값이 많이 올라서 걱정이야 그러게 . . . 서울도 서울인데 창원도 요새...,일반 대화
856,이제 백신접종 차까지 마친 사람들은 시까지 식당이나 술집을 이용할 수 있다네 그래 ...,일반 대화
7272,아 자격증 공부 해야 하는디 요즘 넘 안 해 훈련도 하나의 교육이라 생각하면 편하지...,일반 대화
11496,오늘 길 가는데 너 선임 만났음 키키 엥 ? 어디서 걔를 만나 ? 키키 영화 보고 ...,일반 대화
10619,제주도 여행 가고 싶어요 이번에 가게 되면 코스를 잘 짜고 싶다 코스도 코슨데 식당...,일반 대화


In [141]:
conversations_class_Casual = preprocessed_casual_data['conversation'].copy()
sample_Casual = pd.DataFrame(conversations_class_Casual)

text = textLine_from_row(sample_Casual)

In [151]:
# sample_Casual.head(10)

In [143]:
py_hanspell_text = py_hanspell_spell_correct(text)

py-hanspell 텍스트 교정 시작
706.9410614967346 초 걸렸습니다.


In [147]:
py_hanspell_text

['',
 '',
 '넌 계속 부산에서 살 거냐 나는 부산 안 떠나고 싶어 ㅠㅠ 부산은 너 싫다는데? 하하 바다 없음 못 살아. . . 빌붙어 살래 키키 오 짝사랑할 만큼 가치 있는 거가 밤바다 좋아하는 인 맞아 나도 바다 없이 못 살아 ㅠㅠ^ 보고 싶을 때 볼 수 있는 게 얼마나 좋아 근데 산도 좋아 여름에 춥고 겨울에 따뜻하고 그래서 부산 안 떠날래. . . 너는? 난 갈 수 있으면 가야지 근데 확실히 서울은 클래스가 다르더라 사람은 크게 놀아야 해 ㅎㅎ 서울 생활 무서워',
 '나 며칠 전에 생치 보고 왔음 오 그거 마블 새 영화? 응 맞아 첫 번째 아시안 히어로! 오 어땠어? 궁금하다 첫 번째 아시안 히어로라니 키키 음 뭐라고 해야 할까 좀 어려운데 키키 꿀노잼 ? 꿀노잼 ? 키키 미묘한 평이네 키키 도대체 어땠길래 캐릭터들은 각자 매력 있는데 스토리가 개연성이 좀 없음 키키 아하 그렇군. . . 아쉽다 좀 잘 만들어주지 그러니까! 개인적으로 블랙 팬서보다 좀 못한 듯 블랙 팬서는 완전 흑인들의 아이콘이 됐잖아 맞아 그래서 생치도 좀 그런 느낌이었으면 했는데 그 정도는 또 아니었나 보네 ㅠㅠ 아쉽다 그래도 캐릭터들은 나름 매력적이어서 괜춘 ! 누가 제일 좋았어? 캐릭터 중에? 누구 나오지 난 양조위 ㅠㅠ 양조위 진짜 너무 멋있더라! 헐 양조위도 나와? 화양연화 그 양조위?',
 '',
 '',
 '이제 백신 접종 차까지 마친 사람들은 시까지 식당이나 술집을 이용할 수 있다네 그래? 백신 안 맞은 사람들은? 백신 안 맞은 사람들은 여전히 시까지 이용 가능 키키 키키 나는 차까지 맞아서 시까지 이용 가능하네 키키 그리고 백신 맞은 사람들은 이제 명까지 모일 수 있다 하대 키키 차 접종까지 한 사람들만? 엉 키키 접종 안 맞은 사람들은 또 여전히 명까지 모일 수 있고 키키 너도 차까지 다 맞았지? 난 아직 월 말에 차 맞는다 키키 차 맞을 때 아팠나? 차는 맞은 지 안 맞았는지도 모르겠던데 키키 차가 아프다 그러대 ㅠㅠ 어 나도 차 맞고 일 동안 아팠다 일 

In [167]:
df_py_casual = pd.DataFrame(py_hanspell_text)
df_py_casual

,0
0,
1,
2,넌 계속 부산에서 살 거냐 나는 부산 안 떠나고 싶어 ㅠㅠ 부산은 너 싫다는데? 하...
3,나 며칠 전에 생치 보고 왔음 오 그거 마블 새 영화? 응 맞아 첫 번째 아시안 히...
4,
...,...
3995,
3996,우리 남편은 병장 때 월급 만 원이었대. . . 내도 그 정도였다 만 원? 요즘 얼...
3997,
3998,나 은근 등산을 좋아함 등산 못하겠더라 평지는 시간도 걷겠어 평지에서 일단 훈련을 ...


In [197]:
### 누락 데이터 count 

df_p_c = df_py_casual.copy()

df_p_c = convert_empty_to_nan(df_p_c)
n = count_empty_rows(df_p_c)
print(n)

1466


In [171]:
dataframe_to_csv(df_py_casual,'C:/Users/ParkYeonwoo/Downloads/filtered_normal_conversation.csv')

DataFrame has been saved as C:/Users/ParkYeonwoo/Downloads/filtered_normal_conversation.csv
File saved as C:/Users/ParkYeonwoo/Downloads/filtered_normal_conversation.csv. You can manually download it from the file system.


## [생략 가능] speller(2) BaseOnPLMmodelET5_spell_correct
- test 해 볼 경우에만 import
- local에서는 실행 완료까지 매우 오래 걸리거나 메모리가 부족하므로 colab으로 옮겨서 실행해야 함 

In [ ]:
# import sentencepiece as spm
# print("SentencePiece 라이브러리가 성공적으로 설치되었습니다.")

In [ ]:
ET5_text = BaseOnPLMmodelET5_spell_correct(text)

In [ ]:
df_ET5_casual = pd.DataFrame(ET5_text)
df_ET5_casual

In [ ]:
### 비교용 코드
### 우선 임시 업로드 